In [3]:
import pandas as pd
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import seaborn as sns
import matplotlib_inline as plot

In [310]:
players = pd.read_csv("final_merged_stats.csv")

In [311]:
players

,Rk,Player,Nation,Pos,Squad_x,Age,Born,MP,Starts,Min,...,PrgC,PrgP,PrgR,xG.1,xAG.1,xG+xAG,npxG.1,npxG+xAG.1,year_y,Squad_y
0,1,Gary Ablett,eng ENG,DF,Everton,26,1965,40,40,"3,600",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,Micky Adams,eng ENG,DF,Southampton,30,1961,38,38,"3,245",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,Neil Adams,eng ENG,MF,Oldham Athletic,26,1965,32,26,"2,454",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4,Tony Adams,eng ENG,DF,Arsenal,25,1966,35,33,"3,005",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5,Derek Allan,NaN,DF,Southampton,17,1974,1,0,11,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17523,576,Nicolò Zaniolo,it ITA,"FW,MF",Aston Villa,24,1999,25,9,839,...,35.0,30.0,60.0,0.28,0.11,0.39,0.28,0.39,NaN,NaN
17524,577,Anass Zaroury,ma MAR,"FW,MF",Burnley,22,2000,6,1,152,...,2.0,2.0,12.0,0.10,0.02,0.12,0.10,0.12,NaN,NaN
17525,578,Oleksandr Zinchenko,ua UKR,DF,Arsenal,26,1996,27,20,"1,722",...,40.0,229.0,58.0,0.03,0.13,0.16,0.03,0.16,NaN,NaN
17526,579,Kurt Zouma,fr FRA,DF,West Ham,28,1994,33,32,"2,838",...,7.0,53.0,1.0,0.06,0.01,0.06,0.06,0.06,NaN,NaN


delete unnecessary columns

In [312]:
del players["Nation"]

In [313]:
del players["Rk"]

In [314]:
del players["Born"]

In [315]:
del players["Matches"]

In [316]:
players= players.rename(columns={"year_x": "year"})

In [317]:
players = players.rename(columns={"Squad_x": "Squad"})

In [318]:
players_df = players.copy()

In [ ]:
players_df.columns

In [ ]:
players_df

In [319]:
players_df['Min'] = pd.to_numeric(players_df['Min'].str.replace(',', ''), errors='coerce')

In [320]:
players_df['G+A'] = players_df['G+A'].fillna(0).astype('int64')

In [321]:
players_df['Gls'] = players_df['Gls'].fillna(0).astype('int64')

In [322]:
players_df['Ast'] = players_df['Ast'].fillna(0).astype('int64')

In [323]:
players_df['CrdR'] = players_df['CrdR'].fillna(0).astype('int64')

In [324]:
players_df['CrdY'] = players_df['CrdY'].fillna(0).astype('int64')

In [325]:
players_df['PrgC'] = players_df['PrgC'].fillna(0).astype('int64')

In [326]:
players_df['PrgP'] = players_df['PrgP'].fillna(0).astype('int64')

In [327]:
players_df['PrgR'] = players_df['PrgR'].fillna(0).astype('int64')

In [328]:
players_df = players_df.rename(columns={"Squad_y": "poy_winner_team"})

In [329]:
players_df = players_df.rename(columns={"year_y": "poy_win_year"})

In [ ]:
players_df.dtypes

In [330]:
columns_drop = [
       'G-PK', 'PK', 'PKatt',
]
players_df.drop(columns=columns_drop, inplace=True)

In [331]:
columns_to_drop = ['xG.1', 'xAG.1', 'xG+xAG', 'npxG.1',
       'npxG+xAG.1']
players_df.drop(columns=columns_to_drop, inplace=True)

In [332]:
drop_cols = [
    'Gls.1', 'Ast.1', 'G+A.1', 'G-PK.1', 'G+A-PK'
]
players_df.drop(columns=drop_cols, inplace=True)

In [333]:
players_df['poy_win_year'] = players_df['poy_win_year'].fillna(0).astype('int64')

next step, remove duplicate names by grouping players that played for multiple clubs together either the same year or different years. SO we want total player stats in a year

In [ ]:
name_counts = players_df["Player"].value_counts()
duplicate_names = name_counts[name_counts > 1]

print(duplicate_names)

players_df.groupby(["Player"]).get_group("James Milner")

players_df.groupby(["Player"])

In [334]:
players_df["name"] = players_df["Player"]

In [335]:
players_df

,Player,Pos,Squad,Age,MP,Starts,Min,90s,Gls,Ast,...,xG,npxG,xAG,npxG+xAG,PrgC,PrgP,PrgR,poy_win_year,poy_winner_team,name
0,Gary Ablett,DF,Everton,26,40,40,3600,40.0,0,2,...,NaN,NaN,NaN,NaN,0,0,0,0,NaN,Gary Ablett
1,Micky Adams,DF,Southampton,30,38,38,3245,36.1,4,3,...,NaN,NaN,NaN,NaN,0,0,0,0,NaN,Micky Adams
2,Neil Adams,MF,Oldham Athletic,26,32,26,2454,27.3,8,2,...,NaN,NaN,NaN,NaN,0,0,0,0,NaN,Neil Adams
3,Tony Adams,DF,Arsenal,25,35,33,3005,33.4,0,0,...,NaN,NaN,NaN,NaN,0,0,0,0,NaN,Tony Adams
4,Derek Allan,DF,Southampton,17,1,0,11,0.1,0,0,...,NaN,NaN,NaN,NaN,0,0,0,0,NaN,Derek Allan
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17523,Nicolò Zaniolo,"FW,MF",Aston Villa,24,25,9,839,9.3,2,0,...,2.6,2.6,1.0,3.6,35,30,60,0,NaN,Nicolò Zaniolo
17524,Anass Zaroury,"FW,MF",Burnley,22,6,1,152,1.7,0,0,...,0.2,0.2,0.0,0.2,2,2,12,0,NaN,Anass Zaroury
17525,Oleksandr Zinchenko,DF,Arsenal,26,27,20,1722,19.1,1,2,...,0.6,0.6,2.5,3.1,40,229,58,0,NaN,Oleksandr Zinchenko
17526,Kurt Zouma,DF,West Ham,28,33,32,2838,31.5,3,0,...,1.8,1.8,0.2,2.0,7,53,1,0,NaN,Kurt Zouma


players_df[players_df["Min"] <= 500]

players_df[players_df["MP"] < 10]

players_df = players_df[players_df["MP"] >= 10]

In [ ]:
name_counts = players_df["Player"].value_counts()
duplicate_names = name_counts[name_counts > 1]

duplicate_names

In [ ]:
players_df.groupby(["Player", "year"]).get_group(("Ryan Giggs", 2004))

In [4]:
regrouped_df = pd.read_csv("grouped_data.csv")

In [5]:
regrouped_df

,Player,Pos,Squad,Age,MP,Starts,Min,90s,Gls,Ast,...,year,season,xG,npxG,xAG,npxG+xAG,PrgC,PrgP,PrgR,poy_winner
0,Aaron Connolly,FW,Brighton,19,24,14,1258,14.0,3,1,...,2019,2019/2020,4.0,4.0,0.5,4.5,21,10,86,0
1,NaN,FW,Brighton,20,17,9,791,8.8,2,1,...,2020,2020/2021,4.0,4.0,0.2,4.2,12,4,58,0
2,NaN,"FW,MF",Brighton,21,4,1,156,1.7,0,0,...,2021,2021/2022,0.4,0.4,0.1,0.5,3,1,7,0
3,Aaron Cresswell,"DF,MF",West Ham,24,38,38,3420,38.0,2,4,...,2014,2014/2015,0.0,0.0,0.0,0.0,0,0,0,0
4,NaN,DF,West Ham,25,37,37,3314,36.8,2,4,...,2015,2015/2016,0.0,0.0,0.0,0.0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17523,NaN,GK,West Ham,34,25,25,2117,23.5,0,0,...,2019,2019/2020,0.0,0.0,0.0,0.0,0,0,0,0
17524,NaN,GK,West Ham,35,35,35,3150,35.0,0,0,...,2020,2020/2021,0.0,0.0,0.0,0.0,0,0,0,0
17525,NaN,GK,West Ham,36,37,37,3330,37.0,0,0,...,2021,2021/2022,0.0,0.0,0.0,0.0,0,0,0,0
17526,NaN,GK,West Ham,37,36,36,3111,34.6,0,0,...,2022,2022/2023,0.0,0.0,0.0,0.0,0,0,0,0


In [6]:
regrouped_df['Player'] = regrouped_df['Player'].fillna(method='ffill')
regrouped_df

C:\Users\redi user\AppData\Local\Temp\ipykernel_17420\980892940.py:1: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  regrouped_df['Player'] = regrouped_df['Player'].fillna(method='ffill')


,Player,Pos,Squad,Age,MP,Starts,Min,90s,Gls,Ast,...,year,season,xG,npxG,xAG,npxG+xAG,PrgC,PrgP,PrgR,poy_winner
0,Aaron Connolly,FW,Brighton,19,24,14,1258,14.0,3,1,...,2019,2019/2020,4.0,4.0,0.5,4.5,21,10,86,0
1,Aaron Connolly,FW,Brighton,20,17,9,791,8.8,2,1,...,2020,2020/2021,4.0,4.0,0.2,4.2,12,4,58,0
2,Aaron Connolly,"FW,MF",Brighton,21,4,1,156,1.7,0,0,...,2021,2021/2022,0.4,0.4,0.1,0.5,3,1,7,0
3,Aaron Cresswell,"DF,MF",West Ham,24,38,38,3420,38.0,2,4,...,2014,2014/2015,0.0,0.0,0.0,0.0,0,0,0,0
4,Aaron Cresswell,DF,West Ham,25,37,37,3314,36.8,2,4,...,2015,2015/2016,0.0,0.0,0.0,0.0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17523,Łukasz Fabiański,GK,West Ham,34,25,25,2117,23.5,0,0,...,2019,2019/2020,0.0,0.0,0.0,0.0,0,0,0,0
17524,Łukasz Fabiański,GK,West Ham,35,35,35,3150,35.0,0,0,...,2020,2020/2021,0.0,0.0,0.0,0.0,0,0,0,0
17525,Łukasz Fabiański,GK,West Ham,36,37,37,3330,37.0,0,0,...,2021,2021/2022,0.0,0.0,0.0,0.0,0,0,0,0
17526,Łukasz Fabiański,GK,West Ham,37,36,36,3111,34.6,0,0,...,2022,2022/2023,0.0,0.0,0.0,0.0,0,0,0,0


results_df = pd.DataFrame(columns=players_df.columns)

# Iterate through each player's data
for player, player_data in players_df.groupby('Player'):
    first_row = True  # Flag to check if it's the first row for the player
    for _, row in player_data.iterrows():
        if first_row:
            results_df = pd.concat([results_df, pd.DataFrame([row])], ignore_index=True)  # Append the first row as it is
            first_row = False
        else:
            # For subsequent rows, set the player name to empty string
            #row['Player'] = ""
            results_df = pd.concat([results_df, pd.DataFrame([row])], ignore_index=True)
results_df['Player'] = results_df['Player'].fillna(method='ffill')

results_df



In [7]:
grouped_df = regrouped_df.copy()

In [8]:
grouped_df.to_csv('Correctly_grouped_data.csv')

In [9]:
grouped_df

,Player,Pos,Squad,Age,MP,Starts,Min,90s,Gls,Ast,...,year,season,xG,npxG,xAG,npxG+xAG,PrgC,PrgP,PrgR,poy_winner
0,Aaron Connolly,FW,Brighton,19,24,14,1258,14.0,3,1,...,2019,2019/2020,4.0,4.0,0.5,4.5,21,10,86,0
1,Aaron Connolly,FW,Brighton,20,17,9,791,8.8,2,1,...,2020,2020/2021,4.0,4.0,0.2,4.2,12,4,58,0
2,Aaron Connolly,"FW,MF",Brighton,21,4,1,156,1.7,0,0,...,2021,2021/2022,0.4,0.4,0.1,0.5,3,1,7,0
3,Aaron Cresswell,"DF,MF",West Ham,24,38,38,3420,38.0,2,4,...,2014,2014/2015,0.0,0.0,0.0,0.0,0,0,0,0
4,Aaron Cresswell,DF,West Ham,25,37,37,3314,36.8,2,4,...,2015,2015/2016,0.0,0.0,0.0,0.0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17523,Łukasz Fabiański,GK,West Ham,34,25,25,2117,23.5,0,0,...,2019,2019/2020,0.0,0.0,0.0,0.0,0,0,0,0
17524,Łukasz Fabiański,GK,West Ham,35,35,35,3150,35.0,0,0,...,2020,2020/2021,0.0,0.0,0.0,0.0,0,0,0,0
17525,Łukasz Fabiański,GK,West Ham,36,37,37,3330,37.0,0,0,...,2021,2021/2022,0.0,0.0,0.0,0.0,0,0,0,0
17526,Łukasz Fabiański,GK,West Ham,37,36,36,3111,34.6,0,0,...,2022,2022/2023,0.0,0.0,0.0,0.0,0,0,0,0


In [404]:
grouped_df = grouped_df.astype({'Min': 'int64'})

conv_cols = [
'Age', 'MP', 'Starts', 'Min', 'Gls',
       'Ast', 'G+A', 'CrdY', 'CrdR', 'PrgC', 'PrgP', 'PrgR', 'poy_win_year'
]
grouped_df[conv_cols] = grouped_df[conv_cols].astype('int64')


In [405]:
grouped_df = grouped_df.astype({'year':'object'})

In [11]:
grouped_df.dtypes

Player         object
Pos            object
Squad          object
Age             int64
MP              int64
Starts          int64
Min             int64
90s           float64
Gls             int64
Ast             int64
G+A             int64
CrdY            int64
CrdR            int64
year            int64
season         object
xG            float64
npxG          float64
xAG           float64
npxG+xAG      float64
PrgC            int64
PrgP            int64
PrgR            int64
poy_winner      int64
dtype: object

In [16]:
grouped_df["Pos"].unique()

array(['FW', 'FW,MF', 'DF,MF', 'DF', 'DF,FW', 'MF', 'MF,FW', 'GK',
       'FW,DF', 'MF,DF'], dtype=object)

In [13]:
incorrect_pos = grouped_df[grouped_df['Pos'] == 0]

In [14]:
grouped_df['Pos'] = grouped_df['Pos'].replace('0', 'DF')

pos_split = grouped_df['Pos'].str.get_dummies(sep=',')
grouped_df = pd.concat([grouped_df, pos_split], axis =1)


In [18]:
grouped_df.to_csv("output_files/correctly_grouped_data.csv")

In [343]:
highest_scoring = grouped_df[grouped_df["Gls"] > 15]